In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.fftpack import fft, ifft

In [297]:
def Window(ts):
    '''This function applies a parabolic window to the signal input'''
    n = len(ts)
    W = np.zeros((n,1))
    #create parabola from 0 to length n
    for j in np.arange(0,n):
        W[j] = 1 - np.power((2*(j + 1)/(n+1)-1),2)
    #multiply parabola and signal to get result  
    result = ts * W
    return result

def Bridge(ts):
    '''This function applies the bridge detrended method to the signal input'''
    n = len(ts)
    #points is the length of the time series from n-1 to 0 (backwords)
    points = np.arange(n-1, -1, -1).reshape(1,n)
    #line connecting first and last point
    line = ((ts[0]-ts[n-1])*points/(n-1))+ts[n-1]
    line = line.transpose()
    #subtract line from time series
    result = ts - line
    return result 

def Linreg(x,y):
    '''This function calculates the slope and correlation coeff of a linear fit
    Result(1): slope
    Result(2): coefficient'''
    #calculate mean of x and y (lg frek and lg Pyy)
    mikro = np.mean(x)
    nu = np.mean(y)
    nu_transposed = (y-nu)[np.newaxis].transpose()
    
  

    Qxy = np.sum((x-mikro)*nu_transposed)
    print(Qxy)

def Spec(ts, p, avg): 
    '''This function performs a spectral analysis of the time series
    where ts is the time series, p is the nth power of 2 where 2^n<=length
    of time series. If avg=0, then spectra for all, if avg=1, then spectra avg.
    result(1): Hurst coefficient
    result(2): correlation coefficient
    result(3): beta from Fourier analysis
    result(4): Hurst coefficient (high freq excluded)
    result(5): correlation coefficient (high freq excluded)
    result(6): beta from Fourier analysis (high freq excluded)'''
    
    f = 1
    n = 2**p
    #compute the discrete fourier transform of time series 
    Y = scipy.fftpack.fft(ts.flatten())
    #divide the conjucate of the FFT by n
    Pyy = Y*np.divide(Y.conj(),n)
    #splice array to only include have first half 
    Pyy = Pyy[0:int(n/2)]
    #clear Y and y arrays
    Y = []
    y = []
    #generate array of frequencies from 0 to 0.5
    frek = f/n*np.arange(1,(n/2)+1)
    #transpose frek to column vector 
    frek = np.reshape(frek, (frek.size,1))
    #initialize empty 1D array
    ff = np.zeros((1,p-1)).flatten()
    pp = np.zeros((1,p-1)).flatten()

    if avg == 1: 
        for i in range(1,p):
            ff[i-1]=np.mean(frek[2**(i-1)-1:2**i-1 ])
            pp[i-1]=np.mean(Pyy[2**(i-1)-1:2**i-1])
        #clear Pyy and frek and set value to pp and ff omitting first index
        Pyy=[]
        Pyy = pp[1:len(pp)]
        frek=[]
        frek=ff[1:len(pp)]
        
    #find where Pyy has values of 0, set those values to 0.00001
    k = np.where(Pyy==0)
    if len(k)!=0:
        for kk in range(1,len(k)+1):
            Pyy[k[kk-1]] = 0.00001
    #take natural log of Pyy and frek
    lgPyy = np.log(Pyy)
    lgfrek = np.log(frek)
    #plt.plot(lgfrek,lgPyy)
    #compute linear regression of lgfrek vs lgPyy plot
    curvefit = Linreg(lgfrek,lgPyy)
    #curvefit: (1): slope from lin regression, (2): correlation coeff from lin regression
    

    
    
def FracTool(ts):
    '''This function will aid time series analysis using the concept 
     of statistical fractals.
     This code is adapted from A. Eke, P. Hermán, J. B. Bassingthwaighte, G. M. Raymond, D. B. Percival, 
     M. Cannon, I. Balla, and C. Ikrényi. Physiological time series: distinguishing 
     fractal noises from motions. Pflügers Archiv European Journal of 
     Physiology,4394):403-415, 2000. 
     Fractool determines: 
     1) The signal class of the time series according to the fGn/fBm model
     2) The Hurst coefficients for the fGn or fBm time series
     The input ts is a time series of a single coloumn of data points
    '''
    assert (len(ts) !=0), 'time series is empty'
    
    #INITIALIZATION 
    H_PSD = -1
    H_Disp = -1
    H_bdSWV = -1
    H_fGn = -1
    H_fBm = -1
    
    #PARAMETERS
    n=len(ts) #length of time series input
    print('Number of timepoints = ', n)
    i = 1
    while n >= 2**i: #calculates the closest 2^i value in n 
        i += 1
    p = i - 1
    tsid = ts[0:2**p] #splices ts to exclue points after 2^i
    print('Number of timepoints after 2^p splice =', 2**p)
    del ts #delete signal before splice
    signal_mean = np.mean(tsid) #get mean of tsid
    print('Mean of time series = ', signal_mean)
    n = len(tsid) #update n to length of tsid
    
    #GET BETA USING lowPSDw,e METHOD
    result = Spec(Bridge(Window(tsid)),p,0)

In [298]:
#Load signal from sample fMRI (SCI_Pilot_08)
fbm_sample = np.loadtxt(fname="Test_Signals/sig_fbm_sample.csv", delimiter =",")
fgn_sample = np.loadtxt(fname="Test_Signals/sig_fGn_sample.csv", delimiter =",")
SSC_fbm_sample = np.loadtxt(fname="Test_Signals/sig_SSC_fBm_sample.csv", delimiter =",")
SSC_fgn_sample = np.loadtxt(fname="Test_Signals/sig_SSC_fgn_sample.csv", delimiter =",")
noclass_sample = np.loadtxt(fname="Test_Signals/sig_noclass_sample.csv", delimiter =",")

#turn 1D array into 2D array and transpose so single column vector 
fbm_sample_T = np.reshape(fbm_sample,(fbm_sample.size, 1))
fgn_sample_T = np.reshape(fgn_sample,(fgn_sample.size, 1))
SSC_fbm_sample_T = np.reshape(SSC_fbm_sample,(SSC_fbm_sample.size, 1))
SSC_fgn_sample_T = np.reshape(SSC_fgn_sample,(SSC_fgn_sample.size, 1))
noclass_sample_T = np.reshape(noclass_sample,(noclass_sample.size, 1))


#functioncall
#FracTool(noclass_sample_T)


#Loads fbm signal of known Hurst coeffecient 
#Hurst coeff is 0.7 and signal has 1024 timepoints
fBm_knownH = np.loadtxt(fname="Test_Signals/sig_fbm_knownH.csv", delimiter =",")
#turn 1D array into 2D array and transpose so single column vector 
fBm_knownH_T = np.reshape(fBm_knownH,(fBm_knownH.size, 1))
#function call
FracTool(fBm_knownH_T)

Number of timepoints =  1024
Number of timepoints after 2^p splice = 1024
Mean of time series =  -23.59470008156738
(-1169.025683532077+0j)


In [112]:
for i in range (1,9+1):
    print(i)

1
2
3
4
5
6
7
8


In [250]:
x = np.arange(1,9)
print(x)
print(x-1)

[1 2 3 4 5 6 7 8]
[0 1 2 3 4 5 6 7]
